# Project 4

In this project, you wil implement the natural language processing and logistic regression techniques that you've used this week. 

#### Step 1: Load the Python libraries that you will need for this project. 

In [3]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression
import os
import nltk
import re
import requests
import urllib
from bs4 import BeautifulSoup
# pip install scrapy
# pip install --upgrade zope2
from scrapy.selector import Selector
from scrapy.http import HtmlResponse

%matplotlib inline

#### Step 2: Read in and examine the scraped data

In [3]:
url = "https://www.usajobs.gov/Search/?keyword=department%20of%20defense"

In [ ]:
number_of_pages = total_results / 25 # 25 = displayed per page
number_of_pages

In [4]:
response = requests.get(url)

# Pull HTML string out of requests
html = response.text
html[:500]

u'\r\n<!DOCTYPE html>\r\n<html lang="en" class="scrollbar">\r\n<head><meta charset="utf-8" /><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" /><title>\r\n\t\r\n    \r\n    USAJOBS - The Federal Government&rsquo;s Official Jobs Site\r\n\r\n\r\n</title>      \r\n       <link href="/Content/Styles/usaj-responsive?v=Wlro1xQyoRuae0CQQXV_1ql5IlgERwvWO5gPIEmCw701" rel="stylesheet"/>\r\n\r\n       <link href="/Content/theme-usaj/bundle?v=erLJ6YjgxUdod610aoJBdWnO1M-mnUwaJcPhhPrBN7k1" rel="stylesheet"/>\r\n\r\n\r\n\t\r\n\t\t<scr'

In [5]:
# The same thing but "absolute" reference
job_titles = Selector(text=html).xpath('//a[@class="jobTitleLink"]/text()').extract()

[u'Administrative Officer',
 u'Office Automation Assistant',
 u'Office Automation Assistant (O/A)',
 u'Office Automation Assistant',
 u'School Support Assistant',
 u'School Support Assistant',
 u'Program Analyst',
 u'Supervisory Contract Specialist',
 u'Deputy General Counsel',
 u'Secretary (OA)',
 u'School Information Assistant (OA)',
 u'Secretary (OA)',
 u'Secretary (OA)',
 u'School Information Assistant (O/A)',
 u'School Support Assistant (O/A)',
 u'School Information Assistant',
 u'Contract Specialist(Procurement Analyst)',
 u'Procurement Technician (OA)',
 u'Physician (Child Psychiatry)',
 u'Professor of Small Business',
 u'Trial Attorney (Contract)',
 u'DIRECTOR OF PROGRAM AND BUDGET, RESEARCH AND DEVELOPMENT SUPPORT ACTIVITY',
 u'Intelligence Specialist',
 u'Office Automation Assistant',
 u'Office Automation Assistant']

In [5]:
r = urllib.urlopen('https://www.usajobs.gov/Search/?keyword=analyst&Location=california&AutoCompleteSelected=false').read()
soup = BeautifulSoup(r)
blocks = soup.find_all('div', class_='jobResult')

def get_data(x):
    columns = range(12)
    info = []
    for i in range(len(x)):
        b = x[i].findAll(['div', 'span', 'a'],text=True)
        c = [str(item.get_text()) for item in b if b.index(item) < 12]
        info.append(c)
    data = pd.DataFrame(info, columns = columns)
    return data

jobs = get_data(blocks)
jobs

,0,1,2,3,4,5,6,7,8,9,10,11
0,"Management and Program Analyst, GS-0343-7/9 (P...",Save Job,More Like This,"$43,684.00 - $69,460.00 / Per Year",\r\n GS-0300/03...,Multiple Locations,4,\r\n 5/9/2016 t...,\r\n FSA-2016-0...,\r\n Department...,\r\n Federal St...,\r\n Full-time ...
1,Operations Research Analyst,Save Job,More Like This,"$84,302.00 - $109,592.00 / Per Year",\r\n GS-1500/15...,"\r\n Sunnyvale,...",\r\n 5/2/2016 t...,\r\n SWH816P8EH...,\r\n Department...,\r\n Defense Co...,\r\n Full Time ...,\r\n All U.S. C...
2,Palace Acquire Intern Operations Research Anal...,Save Job,More Like This,"$48,403.00 - $64,067.00 / Per Year",\r\n GS-1500/15...,Multiple Locations,6,\r\n 11/1/2015 ...,\r\n BE16-EHA-1...,\r\n Department...,\r\n Air Force ...,\r\n Full Time ...
3,Operations Research Analyst,Save Job,More Like This,"$32,318.00 - $152,593.00 / Per Year",\r\n GS-1500/15...,Multiple Locations,38,\r\n 5/1/2016 t...,\r\n AFPC-ACQEH...,\r\n Department...,\r\n Air Force ...,\r\n Multiple S...
4,Palace Acquire Intern Operations Research Analyst,Save Job,More Like This,"$39,570.00 - $52,374.00 / Per Year",\r\n GS-1500/15...,Multiple Locations,6,\r\n 11/1/2015 ...,\r\n BE16-EHA-1...,\r\n Department...,\r\n Air Force ...,\r\n Full Time ...
5,Program Analyst,Save Job,More Like This,"$52,505.00 - $68,252.00 / Per Year",\r\n GS-0300/03...,\r\n Sacramento...,\r\n 4/27/2016 ...,\r\n 16-156 (16...,\r\n Department...,\r\n Army Natio...,\r\n Full Time ...,\r\n Current Ca...
6,"Operations Research Analyst, DR-1515-2/3/4",Save Job,More Like This,"$61,289.00 - $133,444.00 / Per Year",\r\n DR-1500/15...,Multiple Locations,9,\r\n 4/1/2016 t...,\r\n 9X16EHA-16...,\r\n Department...,\r\n Air Force ...,\r\n Full Time ...
7,FINANCIAL MANAGEMENT ANALYST,Save Job,More Like This,"$79,272.00 - $103,053.00 / Per Year",\r\n GS-0500/05...,\r\n Seal Beach...,\r\n 5/9/2016 t...,\r\n SE0501-12-...,\r\n Department...,\r\n Naval Faci...,\r\n Full Time ...,\r\n United Sta...
8,"Contract Spec/Administrator/Negotiator, Procur...",Save Job,More Like This,"$51,163.00 - $158,700.00 / Per Year",\r\n GS-1100/11...,Multiple Locations,14,\r\n 11/2/2015 ...,\r\n 1612005-DH...,\r\n General Se...,\r\n General Se...,"\r\n Full-time,..."
9,Contract Price/Cost Analyst,Save Job,More Like This,"$38,366.00 - $61,783.00 / Per Year",\r\n GS-1100/11...,"\r\n Stockton, ...",\r\n 5/2/2016 t...,\r\n SWH816KSEH...,\r\n Department...,\r\n Defense Co...,\r\n Full Time ...,\r\n All U.S. C...


In [8]:
next_page = Selector(text=html).xpath("//a[@class='nextPage']/@href").extract()
url + next_page[0]

u'https://www.usajobs.gov/Search/?keyword=department%20of%20defense/Search/GetPageResults/?page=2&statusFilter=public'

In [82]:
for i in range(3):
#     print next_titles
    print next_page[0]
    print '------------------------------'
    url = url + next_page[0]
    next_response = requests.get(url)
    next_html = next_response.text
    next_page = Selector(text=next_html).xpath('//a[@class="nextPage"]/@href').extract()
    next_titles = Selector(text=next_html).xpath('//a[@class="jobTitleLink"]/text()').extract()

IndexError: list index out of range

In [87]:
url + next_page[0]

u'https://www.usajobs.gov/Search/?keyword=department%20of%20defense/Search/GetPageResults/?page=2&statusFilter=public'

In [91]:
next_response = requests.get(url + next_page[0])
next_html = next_response.text

print next_html
# test = Selector(text=next_html).xpath('//a[@title="Go To Next Page"]/@href').extract()

# test


<!DOCTYPE html>
<html lang="en" class="scrollbar">
<head><meta charset="utf-8" /><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" /><title>
	
    
    USAJOBS - The Federal Government&rsquo;s Official Jobs Site


</title>      
       <link href="/Content/Styles/usaj-responsive?v=Wlro1xQyoRuae0CQQXV_1ql5IlgERwvWO5gPIEmCw701" rel="stylesheet"/>

       <link href="/Content/theme-usaj/bundle?v=erLJ6YjgxUdod610aoJBdWnO1M-mnUwaJcPhhPrBN7k1" rel="stylesheet"/>


	
		<script type="text/javascript">
		    var lis = false;
		    var isEagle = false;
		    var myInactivity = 1620000;
		    var mySessionTimeout = 1800000;
		    var acHostUrl =  'https://ac.usajobs.gov/acwd';
		    var acAgencyHostUrl =  'https://ac.usajobs.gov/agencyAC';
		    var usajobsConfig = {"DataHome":"https://data.usajobs.gov/","MediaHome":"https://media.usajobs.gov/","BingKey":"Aq2HseksGy7yVZIw7TqmI6AQrgSsye0gPEVVGauJnnP1Zz9aGAgnVFjBERPX5otI","CanSeekStatusJobs":false};
		</script>
		<script src="/bundles/

In [ ]:
for i in range(6):
    print next_link[0]
    print '------------------------------'
    next_response = requests.get("http://www.datatau.com" + next_link[0])
    next_HTML = next_response.text
    next_link = Selector(text=next_HTML).xpath('//a[text()="More"]/@href').extract()
    next_points = Selector(text=next_HTML).xpath('//td[@class="subtext"]/span/text()').extract()
    print next_points

In [39]:
next_page

[]

In [6]:
jobs = pd.read_csv("your_data_here")
jobs.head()

IOError: File 
<!DOCTYPE html>
<html lang="en" class="scrollbar">
<head><meta charset="utf-8" /><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" /><title>
	
    
    USAJOBS - The Federal Government&rsquo;s Official Jobs Site


</title>      
       <link href="/Content/Styles/usaj-responsive?v=Wlro1xQyoRuae0CQQXV_1ql5IlgERwvWO5gPIEmCw701" rel="stylesheet"/>

       <link href="/Content/theme-usaj/bundle?v=erLJ6YjgxUdod610aoJBdWnO1M-mnUwaJcPhhPrBN7k1" rel="stylesheet"/>


	
		<script type="text/javascript">
		    var lis = false;
		    var isEagle = false;
		    var myInactivity = 1620000;
		    var mySessionTimeout = 1800000;
		    var acHostUrl =  'https://ac.usajobs.gov/acwd';
		    var acAgencyHostUrl =  'https://ac.usajobs.gov/agencyAC';
		    var usajobsConfig = {"CanSeekStatusJobs":false,"DataHome":"https://data.usajobs.gov/","MediaHome":"https://media.usajobs.gov/","BingKey":"Aq2HseksGy7yVZIw7TqmI6AQrgSsye0gPEVVGauJnnP1Zz9aGAgnVFjBERPX5otI"};

### Intro: Write a problem statement / aim for this project

Hint: We want to understand which attributes affect salaries in data science

### Part 1: Data Wrangling

#### 1. Delete the unnecessary rows in the data

#### 1.1 Rename the columns for easier reading

#### 1.2 Tokenize the position's descriptions

#### 1.3 Search for position descriptions that include "data"

Search the tokenized sentances for "data"; Use '1' if the description has "data" and '0' if it doesn't.  **Hint**: Write a function and loop through the *description* column

Apply the function and delete the old "description" column

#### 1.4: Wrangle the *salary* data and compute a mean salary for each position

Break Apart the Salary Range into two columns 

and join the new ranges to the original dataframe

Convert the new ranges back to numeric

Calculate the Mean Salary and append it to the Dataframe

## Part 2: Exploratory Data Analysis

#### 1. Convert the Categorical Data to Numeric for EDA

In [ ]:
def workclass_to_numeric(x):
    if x=='Philadelphia, Pennsylvania':
        return 1
    if x=='Concord, Massachusetts':
        return 2
    if x=='Greenbelt, Maryland':
        return 3
    if x=='Sacramento, California':
        return 4
    if x=='Oakland, California':
        return 5
    if x=='Pullman, Washington':
        return 6

    
Etc.

#### 1.2 Run a basic description of the data

#### 1.3 Plot the Data

## Part 3: Logistic Regression

#### 1. Define the variables that we will use in our regression analysis

#### 1.2 Transform "Y" into a 1-Dimensional Array for SciKit-Learn

#### 1.3 Conduct the Regression

#### 1.4 How can we judge the accuracy of this analysis? 

#### 1.5 Examine the coefficients to see our correlations